In [13]:
import re
import math
import random
from typing import List, Dict, Tuple, Callable, Optional

import numpy as np
import pandas as pd


# =============================
# 0) НАСТРОЙКИ
# =============================
CSV_IN = "phase5_plan_detailed_hours.csv"   # <-- твой вход
CSV_OUT = "schedule_best.csv"

ROLES = ["BE_M", "BE_J", "FE_J", "QA", "PM"]
ROLE_H_COLS = ["BE_M_h", "BE_J_h", "FE_J_h", "QA_h", "PM_h"]

HOURS_PER_DAY = 8.0

# Сколько людей в каждой роли (у тебя по 1 каждого)
TEAM_SIZE = {"BE_M": 1, "BE_J": 1, "FE_J": 1, "QA": 1, "PM": 1}

# Эвристики + сколько случайных прогонов
N_RANDOM = 500
SEED = 42


# =============================
# 1) ЧТЕНИЕ CSV + ПАРСИНГ ПРЕДШЕСТВЕННИКОВ
# =============================
def parse_predecessors(cell) -> List[int]:
    """
    Поддерживает:
    - пусто/NaN/'-'/ '—'/ '0' -> []
    - '3|4|10', '3,4,10', '3 4 10', '3;4;10' -> [3,4,10]
    - игнорирует 0
    """
    if cell is None:
        return []
    if isinstance(cell, float) and math.isnan(cell):
        return []
    s = str(cell).strip()
    if s == "" or s in {"-", "—", "–", "None", "NONE", "null", "NULL"}:
        return []
    nums = [int(x) for x in re.findall(r"\d+", s)]
    nums = [x for x in nums if x != 0]
    return nums


df = pd.read_csv(CSV_IN)
df.columns = [c.strip() for c in df.columns]

required = {"TaskID", "TaskName", "Duration_days", "Predecessors"} | set(ROLE_H_COLS)
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"CSV missing columns: {missing}. Found: {list(df.columns)}")

# сортируем по TaskID для стабильности
df["TaskID"] = pd.to_numeric(df["TaskID"], errors="raise").astype(int)
df = df.sort_values("TaskID").reset_index(drop=True)

task_ids = df["TaskID"].tolist()
id2idx: Dict[int, int] = {tid: i for i, tid in enumerate(task_ids)}
n = len(task_ids)

dur = pd.to_numeric(df["Duration_days"], errors="raise").astype(int).tolist()
if any(d <= 0 for d in dur):
    bad = [(task_ids[i], dur[i]) for i in range(n) if dur[i] <= 0]
    raise ValueError(f"Duration_days must be positive. Bad: {bad}")

pred_ids: List[List[int]] = [parse_predecessors(x) for x in df["Predecessors"].tolist()]

# проверка: все предшественники существуют
broken = []
for i in range(n):
    for p in pred_ids[i]:
        if p not in id2idx:
            broken.append((task_ids[i], p))
if broken:
    msg = "Broken predecessors (Task depends on missing TaskID):\n" + "\n".join(
        [f"  Task {t} -> missing {p}" for t, p in broken]
    )
    raise ValueError(msg)

# переводим pred IDs -> pred индексы
pred: List[List[int]] = [[id2idx[p] for p in ps] for ps in pred_ids]

# successors
succ: List[List[int]] = [[] for _ in range(n)]
for j in range(n):
    for p in pred[j]:
        succ[p].append(j)


# =============================
# 2) DEMAND PER DAY (часы/день на роль)
# =============================
# часы по ролям
role_hours = df[ROLE_H_COLS].apply(pd.to_numeric, errors="coerce").fillna(0.0).values  # shape (n, 5)

# demand_per_day[i][r] = role_hours[i][r] / dur[i]
demand = np.zeros((n, len(ROLES)), dtype=float)
for i in range(n):
    demand[i, :] = role_hours[i, :] / float(dur[i])

# capacity per day (часы/день на роль) = team_size * 8
capacity = np.array([TEAM_SIZE[r] * HOURS_PER_DAY for r in ROLES], dtype=float)


# =============================
# 3) CPM: ES, LF, TF, FF (без ресурсов)
# =============================
def topo_order(pred: List[List[int]]) -> List[int]:
    remaining = [set(p) for p in pred]
    ready = [i for i in range(n) if not remaining[i]]
    order = []

    while ready:
        i = ready.pop(0)
        order.append(i)
        for s in succ[i]:
            remaining[s].discard(i)
            if not remaining[s]:
                ready.append(s)

    if len(order) != n:
        raise ValueError("Cycle detected in precedence constraints")
    return order


order = topo_order(pred)

# ES
es = [0] * n
for i in order:
    es[i] = max((es[p] + dur[p] for p in pred[i]), default=0)

makespan_cpm = max(es[i] + dur[i] for i in range(n))

# LF
lf = [makespan_cpm] * n
for i in reversed(order):
    if succ[i]:
        lf[i] = min(lf[s] - dur[s] for s in succ[i])
    else:
        lf[i] = makespan_cpm

tf = [lf[i] - es[i] - dur[i] for i in range(n)]
ff = []
for i in range(n):
    if succ[i]:
        ff.append(min(es[s] for s in succ[i]) - es[i] - dur[i])
    else:
        ff.append(0)


# =============================
# 4) Activity list generator (topological, with a pick rule)
# =============================
def generate_activity_list(pick: Optional[Callable[[List[int]], int]] = None) -> List[int]:
    remaining = [set(p) for p in pred]
    ready = [i for i in range(n) if not remaining[i]]
    alist = []

    while ready:
        i = pick(ready) if pick else random.choice(ready)
        ready.remove(i)
        alist.append(i)
        for s in succ[i]:
            remaining[s].discard(i)
            if not remaining[s]:
                ready.append(s)

    if len(alist) != n:
        raise ValueError("Failed to generate activity list (cycle?)")
    return alist


# =============================
# 5) SSGS decoder (дискретные дни)
# =============================
def earliest_start_due_to_pred(i: int, starts: List[int]) -> int:
    if not pred[i]:
        return 0
    return max(starts[p] + dur[p] for p in pred[i])

def fits(i: int, t0: int, usage: np.ndarray) -> bool:
    """Проверка ресурсов на интервале [t0, t0+dur[i])"""
    for t in range(t0, t0 + dur[i]):
        if np.any(usage[t, :] + demand[i, :] - capacity > 1e-9):
            return False
    return True

def apply(i: int, t0: int, usage: np.ndarray) -> None:
    for t in range(t0, t0 + dur[i]):
        usage[t, :] += demand[i, :]

def decode_ssgs(alist: List[int]) -> Tuple[List[int], int]:
    """
    Serial Schedule Generation Scheme:
    идём по activity list, ставим каждую задачу в самое раннее время,
    где выполнены предшественники и хватает ресурсов.
    """
    # верхняя оценка горизонта: сумма длительностей
    H = sum(dur) + 5
    usage = np.zeros((H, len(ROLES)), dtype=float)
    starts = [0] * n

    for i in alist:
        t = earliest_start_due_to_pred(i, starts)
        while True:
            if t + dur[i] >= H:
                # расширим горизонт
                H_new = int(H * 1.5) + 10
                usage2 = np.zeros((H_new, len(ROLES)), dtype=float)
                usage2[:H, :] = usage
                usage = usage2
                H = H_new

            if fits(i, t, usage):
                starts[i] = t
                apply(i, t, usage)
                break
            t += 1

    ms = max(starts[i] + dur[i] for i in range(n))
    return starts, ms


# =============================
# 6) Эвристики
# =============================
def rule_slk(i): return tf[i]              # min
def rule_free(i): return ff[i]            # min
def rule_lst(i): return lf[i] - dur[i]    # min
def rule_lft(i): return lf[i]             # min
def rule_lstlft(i): return (lf[i] - dur[i]) + lf[i]  # min

def rule_grpw(i): return dur[i] + sum(dur[s] for s in succ[i])  # max
def rule_lpt(i): return dur[i]                                 # max
def rule_mis(i): return len(succ[i])                            # max
def rule_grd(i): return dur[i] * float(np.sum(demand[i, :]))     # max
def rule_grwc(i): return float(np.sum(demand[i, :]))             # max
def rule_gcrwc(i): return float(np.sum(demand[i, :])) + sum(float(np.sum(demand[s, :])) for s in succ[i])  # max
def rule_rot(i):
    if dur[i] == 0:
        return 0.0
    ratio = 0.0
    for r in range(len(ROLES)):
        if capacity[r] > 0:
            ratio += demand[i, r] / capacity[r]
    return ratio / dur[i]                                       # max

HEURISTICS: List[Tuple[str, Callable[[int], float], str]] = [
    ("SLK", rule_slk, "min"),
    ("FREE", rule_free, "min"),
    ("LST", rule_lst, "min"),
    ("LFT", rule_lft, "min"),
    ("LSTLFT", rule_lstlft, "min"),
    ("GRPW", rule_grpw, "max"),
    ("LPT", rule_lpt, "max"),
    ("MIS", rule_mis, "max"),
    ("GRD", rule_grd, "max"),
    ("GRWC", rule_grwc, "max"),
    ("GCRWC", rule_gcrwc, "max"),
    ("ROT", rule_rot, "max"),
]


# =============================
# 7) Поиск лучшего расписания
# =============================
random.seed(SEED)
np.random.seed(SEED)

best = None  # (makespan, name, starts, alist)

def make_pick(rule, mode):
    if mode == "min":
        return lambda ready: min(ready, key=rule)
    else:
        return lambda ready: max(ready, key=rule)

# deterministic heuristics
for name, rule, mode in HEURISTICS:
    alist = generate_activity_list(pick=make_pick(rule, mode))
    starts, ms = decode_ssgs(alist)
    if best is None or ms < best[0]:
        best = (ms, name, starts, alist)

# random
for _ in range(N_RANDOM):
    alist = generate_activity_list(pick=None)
    starts, ms = decode_ssgs(alist)
    if ms < best[0]:
        best = (ms, "RANDOM", starts, alist)

best_ms, best_name, best_starts, best_alist = best

print("=== BEST SCHEDULE ===")
print("Best heuristic:", best_name)
print("Makespan (working days):", best_ms)
print("Working weeks (5d/week):", round(best_ms / 5.0, 2))
print("~ months (22d/month):   ", round(best_ms / 22.0, 2))
print()


results = []

# 1) deterministic heuristics — пересчитаем и соберём результаты
for name, rule, mode in HEURISTICS:
    alist = generate_activity_list(pick=make_pick(rule, mode))
    starts, ms = decode_ssgs(alist)
    results.append({
        "Heuristic": name,
        "Type": mode,
        "Makespan_days": int(ms),
        "Makespan_weeks": round(ms / 5.0, 2),
    })

# 2) random runs — соберём статистику
random_makespans = []
for _ in range(N_RANDOM):
    alist = generate_activity_list(pick=None)
    starts, ms = decode_ssgs(alist)
    random_makespans.append(ms)

random_makespans = np.array(random_makespans, dtype=float)
results.append({
    "Heuristic": f"RANDOM (best of {N_RANDOM})",
    "Type": "random",
    "Makespan_days": int(np.min(random_makespans)),
    "Makespan_weeks": round(float(np.min(random_makespans)) / 5.0, 2),
})
results.append({
    "Heuristic": f"RANDOM (median of {N_RANDOM})",
    "Type": "random",
    "Makespan_days": int(np.median(random_makespans)),
    "Makespan_weeks": round(float(np.median(random_makespans)) / 5.0, 2),
})
results.append({
    "Heuristic": f"RANDOM (p80 of {N_RANDOM})",
    "Type": "random",
    "Makespan_days": int(np.percentile(random_makespans, 80)),
    "Makespan_weeks": round(float(np.percentile(random_makespans, 80)) / 5.0, 2),
})

res_df = pd.DataFrame(results).sort_values("Makespan_days").reset_index(drop=True)

print("\n=== ALL HEURISTICS RESULTS ===")
print(res_df.to_string(index=False))

# Красивый вывод победителя
best_row = res_df.iloc[0]
print("\nBEST OVERALL:", best_row["Heuristic"], "->", best_row["Makespan_days"], "days",
      f"({best_row['Makespan_weeks']} weeks)")

# =============================
# 8) Экспорт результата
# =============================
out_rows = []
for i in range(n):
    out_rows.append({
        "TaskID": task_ids[i],
        "TaskName": df.loc[i, "TaskName"],
        "Start_day": int(best_starts[i]),
        "End_day": int(best_starts[i] + dur[i]),
        "Duration_days": int(dur[i]),
        "Predecessors": df.loc[i, "Predecessors"],
        **{f"{ROLES[r]}_h_total": float(role_hours[i, r]) for r in range(len(ROLES))}
    })

out = pd.DataFrame(out_rows).sort_values("Start_day").reset_index(drop=True)
out.to_csv(CSV_OUT, index=False, encoding="utf-8-sig")

print(f"Saved: {CSV_OUT}")
print()
print(out[["TaskID", "TaskName", "Start_day", "End_day", "Duration_days"]].to_string(index=False))


=== BEST SCHEDULE ===
Best heuristic: SLK
Makespan (working days): 55
Working weeks (5d/week): 11.0
~ months (22d/month):    2.5


=== ALL HEURISTICS RESULTS ===
             Heuristic   Type  Makespan_days  Makespan_weeks
                   SLK    min             55            11.0
                   LST    min             55            11.0
                   LFT    min             55            11.0
                LSTLFT    min             55            11.0
  RANDOM (best of 500) random             55            11.0
                  FREE    min             57            11.4
                  GRWC    max             60            12.0
                   ROT    max             60            12.0
RANDOM (median of 500) random             61            12.2
                   MIS    max             61            12.2
                  GRPW    max             61            12.2
                 GCRWC    max             61            12.2
                   LPT    max             63 